In [1]:
%reload_ext autoreload
%autoreload 2
#use gpu 1 only
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import json
from pathlib import Path
from typing import Iterator
import time
import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,  
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)      
    
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 
from accelerate import Accelerator

import torch.nn.functional as F
# from einops import rearrange, pack, unpack
import sys
sys.path.append('/home/kiki/dados_servidor/Servidor/LM_Pretraining/EfficientPretrain/')

In [2]:
model_training='hf'#'hf' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 100_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable

# VOCAB_SIZE = 32_768  # from Cramming
# DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
# MODEL_MAX_SEQ_LEN = 128  # from Cramming

# VOCAB_SIZE = 16_000  # tokenmonster
VOCAB_SIZE = 32_768  # bert
DEVICE_BATCH_SIZE = 12 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
# MODEL_MAX_SEQ_LEN = 84*2 # token_monster
MODEL_MAX_SEQ_LEN = 512 # bert


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from utils.process_tokenizer import get_tokenizer
tokenized_dataset,norm,vocab,tokenizer=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'smlm',MODEL_MAX_SEQ_LEN+2,model_training)
# MASK_ID=vocab.tokenize("[MASK]")[0]
N_DOMAINS=3


batch_size:  2040


Found cached dataset parquet (/home/kiki/.cache/huggingface/datasets/sradc___parquet/sradc--chunked-shuffled-wikipedia20220301en-bookcorpusopen-ff5cb88917a65ec5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


loaded dataset!!


Feeding samples to tokenizer: 100%|██████████| 100000/100000 [00:05<00:00, 19903.17it/s]





Tokenizer trained in 7.9 seconds.


In [3]:
from x_transformers import TransformerWrapper, Decoder,Encoder


#encoder decoder
colt5_dict={
            'encoder':{'is_colt5_encoder':True,'light_ff_mult':0.5,'heavy_ff_mult':4,'num_heavy_tokens':25,
                   'light_dim_head':64,'light_heads':8,'light_window_size':128,
                   'heavy_dim_head':64,'heavy_heads':8,'num_heavy_tokens_q':25,
                   'num_heavy_tokens_kv':25},
            }

# model = TransformerWrapper(
base_model = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    attn_layers = Encoder(
        dim = int(768),
        depth = 12,
        heads = 12,
        attn_flash = True,
        colt5_dict=colt5_dict['encoder'],
        # rel_pos_bias = True
    ),
    emb_dim=128,
)

class SpanBert(nn.Module):
    def __init__(self, model, max_targets=20, position_embedding_size=200,num_samples=5):
        super().__init__()
        self.model = model
        self.position_embeddings = nn.Embedding(max_targets, position_embedding_size)
        model_dim = self.model.emb_dim
        self.mlp_layer_norm = nn.LayerNorm(768 * 2 + position_embedding_size)
        self.decoder = nn.Linear(768 * 2 + position_embedding_size, VOCAB_SIZE+8,bias=False)
        self.bias = nn.Parameter(torch.zeros(VOCAB_SIZE+8))
        self.max_targets = max_targets
        self.num_samples=num_samples

    def forward(self, x,span_starts,span_ends):
        out,hidden_states=self.model(x,return_logits_and_embeddings=True)
            
        # print('out.shape: ',out.shape)
        # print('span_starts: ',span_starts)
        # print('span_ends: ',span_ends)  
        # print('hidden_states.shape: ',hidden_states.shape)

        #get external boundary tokens indices(before MASK_ID occurs and after it)
        indexes=[]
        pairs=[]
        for i in range(len(span_starts)):
            for j in range(len(span_starts[i])):
                s=span_starts[i][j]-1
                e=span_ends[i][j] + 1
                if e>=len(x[i]):
                    e=len(x[i])-1
                if s<0:
                    s=0
                start_emb = hidden_states[i,s]
                end_emb = hidden_states[i,e]
                for k in range(s,e):
                    pairs.append(torch.cat((start_emb,end_emb,self.position_embeddings(torch.tensor(k-s).to(x.device))),-1))
                
        # print('pairs',len(pairs))
        #get 5 only
        # pairs=torch.stack(pairs[:self.num_samples])
        pairs=torch.stack(pairs[:])
        # print('pairs',pairs.shape)
        p=self.mlp_layer_norm(pairs)
        # print('p: ',p.shape)
        d=self.decoder(p) + self.bias
        #get argmax
        #d=torch.argmax(d,-1)
        #reshape d from (a,b) to (1,a,b)
        d=torch.unsqueeze(d,0)
        #print('d: ',d.shape)
        return out,d

model = SpanBert(base_model, max_targets=20, position_embedding_size=200)

In [4]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils.utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [5]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):

        # with accelerator.accumulate(model):
        spans_start=batch['spans_start']
        spans_end=batch['spans_end']
        spans_start=[[int(j) for j in i.split()] for i in spans_start]
        spans_end=[[int(j) for j in i.split()] for i in spans_end]    

        logits_mlm,logits_span=model(batch['input_ids'],spans_start,spans_end) 
        
        span_tokens=[]
        for i in range(len(spans_start)):
            for j in range(len(spans_start[i])):
                s=spans_start[i][j] - 1
                e=spans_end[i][j] + 1
                if e>=len(batch['input_ids'][i]):
                    e=len(batch['input_ids'][i])-1
                if s<0:
                    s=0
                for k in range(s,e):
                    span_tokens.append(batch['unmasked_tokens'][i][k])
        span_tokens=torch.tensor(span_tokens).to(batch['input_ids'].device)
        span_tokens=torch.unsqueeze(span_tokens,0)
        #print('span_tokens: ',span_tokens.shape)
        #print(batch['unmasked_tokens'].shape)
        #print(logits_mlm.transpose(1, 2).shape)
        #print(logits_span.transpose(1, 2).shape)
        loss_mlm = F.cross_entropy(logits_mlm.transpose(1, 2),batch['unmasked_tokens'],ignore_index = PAD_ID)
        loss_span = F.cross_entropy(logits_span.transpose(1, 2),span_tokens,ignore_index = PAD_ID)
        
        loss=loss_mlm+loss_span
        count_amostra+=int(len(batch['input_ids']))
        #print(loss)
        
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1
    if completed_steps >= max_train_steps:
        break
    model.eval()
    losses = [] 
    

iteration:  0 , total_loss:  0.7041753133138021
count_amostra: 12
iteration:  30 , total_loss:  18.61679318745931
count_amostra: 372


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:22                                                                                   │
│                                                                                                  │
│   19 │   │   spans_start=[[int(j) for j in i.split()] for i in spans_start]                      │
│   20 │   │   spans_end=[[int(j) for j in i.split()] for i in spans_end]                          │
│   21 │   │                                                                                       │
│ ❱ 22 │   │   logits_mlm,logits_span=model(batch['input_ids'],spans_start,spans_end)              │
│   23 │   │                                                                                       │
│   24 │   │   span_tokens=[]                                                                      │
│   25 │   │   for i in range(len(spans_start)):                                                   │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/nn/modules/module.py:1501 in  │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ in forward:40                                                                                    │
│                                                                                                  │
│   37 │   │   self.num_samples=num_samples                                                        │
│   38 │                                                                                           │
│   39 │   def forward(self, x,span_starts,span_ends):                                             │
│ ❱ 40 │   │   out,hidden_states=self.model(x,return_logits_and_embeddings=True)                   │
│   41 │   │                                                                                       │
│   42 │   │   # print('out.shape: ',out.shape)                                                    │
│   43 │   │   # print('span_starts: ',span_starts)                                                │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/nn/modules/module.py:1501 in  │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used    

In [6]:
torch_model=model.model

In [7]:

#convert to huggingface Transformers

from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

config = BertConfig(
    vocab_size=VOCAB_SIZE+8,
    max_position_embeddings=MODEL_MAX_SEQ_LEN,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=2,
)

bert_model=BertForMaskedLM(config=config)
bert_model.bert=torch_model

In [8]:
bert_model.save_pretrained('models/bert_spanbert')

In [9]:
# 1m39s
# 3m 27s sequence of 256

## Evaluate

### QA

### Trainer hf

In [10]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)

Found cached dataset squad (/home/kiki/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [17]:
max_length = MODEL_MAX_SEQ_LEN#384
stride = 128
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [13]:
validation_dataset = dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=tokenizer.model_max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # print('sequence_ids: ',sequence_ids)
        # print('sequence_ids: ',len(sequence_ids))

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [18]:
from transformers import DefaultDataCollator

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names,batch_size=4)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#BertForMaskedLM
from transformers import BertForMaskedLM

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)
# bert_model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path='bert-base-uncased')
# bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert_spanbert")


Some weights of the model checkpoint at /home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert_spanbert were not used when initializing BertForQuestionAnswering: ['bert.attn_layers.layers.4.1.q_router.routing_token', 'bert.attn_layers.layers.5.1.light_ff.1.weight', 'bert.attn_layers.layers.6.1.heavy_attn.norm.weight', 'bert.attn_layers.final_norm.bias', 'bert.attn_layers.layers.5.1.light_ff.1.bias', 'cls.predictions.transform.dense.bias', 'bert.attn_layers.layers.0.1.heavy_attn.to_out.weight', 'bert.attn_layers.layers.4.1.light_attn.norm.weight', 'bert.attn_layers.layers.5.1.light_ff.3.weight', 'bert.attn_layers.layers.4.1.light_attn.to_qkv.weight', 'bert.attn_layers.layers.1.1.heavy_ff.1.weight', 'bert.attn_layers.layers.1.1.light_ff.1.weight', 'bert.project_emb.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.attn_layers.layers.0.1.kv_router.routing_token', 'bert.attn_layers.layers.4.1.heavy_attn.null_kv', 'bert.attn_layers.layers.2.1.light_attn.to_out.weight', 'bert.att

In [17]:
trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)
trainer.train()

/home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: abcp4. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.757800


TrainOutput(global_step=500, training_loss=4.7578427734375, metrics={'train_runtime': 138.6365, 'train_samples_per_second': 28.852, 'train_steps_per_second': 3.607, 'total_flos': 1045187026944000.0, 'train_loss': 4.7578427734375, 'epoch': 1.0})

In [18]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [19]:
import evaluate

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["test"])

  0%|          | 0/1000 [00:00<?, ?it/s]

{'exact_match': 1.8, 'f1': 7.352985910764617}

### Custom

In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MODEL_MAX_SEQ_LEN,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # print('sequence_ids: ',sequence_ids)
        # print('sequence_ids: ',len(sequence_ids))

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [10]:
len(dataset)

2

In [11]:
from transformers import DefaultDataCollator

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names,batch_size=4)

data_collator = DefaultDataCollator()

#data loader
train_dataloader = DataLoader(
        tokenized_dataset['train'], shuffle=True, batch_size=DEVICE_BATCH_SIZE,collate_fn=data_collator
    )

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
tokenized_dataset['test']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1000
})

In [13]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [14]:
model_extractor = HiddenLayerExtractor(model.model, -2)

In [15]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.s = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN), 768)
        self.e = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN), 768)
        
    
    def forward(self, ids):
        words_embs= self.l1(ids)
        # print('l1:',words_embs.shape)
        output = self.l2(words_embs)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        s_output = self.s(output)
        e_output = self.e(output)
        return s_output,e_output,words_embs

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=5e-5,weight_decay=0.01)

In [16]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs.to('cpu'), targets.to('cpu'))
    return loss

def get_index(s_embs,e_embs,words_embs,logits_only=False):
    s_index=torch.bmm(words_embs,s_embs.unsqueeze(2)).squeeze(2)
    e_index=torch.bmm(words_embs,e_embs.unsqueeze(2)).squeeze(2)
    if logits_only:
        return s_index,e_index

    s_index=torch.softmax(s_index,dim=1)
    e_index=torch.softmax(e_index,dim=1)
    return s_index,e_index

In [17]:
#FINETUNING
epochs=5
running_loss=0
for i in range(epochs):
    for j,data in enumerate(train_dataloader):
        input_ids=torch.tensor(data['input_ids']).to('cuda', dtype = torch.long)
        spans_start = data['start_positions']
        spans_end = data['end_positions']

        s_embs,e_embs,words_embs = classifier_model(input_ids)
        s_index,e_index=get_index(s_embs,e_embs,words_embs)
        start_loss = loss_fn(s_index,spans_start)
        end_loss = loss_fn(e_index,spans_end)        
        
        loss=(start_loss+end_loss)/2
        running_loss+=loss.item()
        if j%50==0:
            # print(s_index.argmax(dim=1))
            # print(e_index.argmax(dim=1))
            # print(targets[:,0])
            # print(targets[:,1])
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break

/tmp/ipykernel_167542/1414205731.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids=torch.tensor(data['input_ids']).to('cuda', dtype = torch.long)


Epoch: 0, Loss:  0.11083119392395019
Epoch: 1, Loss:  0.11102809906005859
Epoch: 2, Loss:  0.11102415084838867
Epoch: 3, Loss:  0.11083263397216797
Epoch: 4, Loss:  0.1101995849609375


In [18]:
# #evaluate
# from tqdm.auto import tqdm
# import evaluate
# import collections
# import numpy as np

# metric = evaluate.load("squad")
# n_best = 20
# max_answer_length = 30
# predicted_answers = []

# def compute_metrics(start_logits, end_logits, features, examples):
#     example_to_features = collections.defaultdict(list)
#     for idx, feature in enumerate(features):
#         example_to_features[feature["example_id"]].append(idx)

#     predicted_answers = []
#     for example in tqdm(examples):
#         example_id = example["id"]
#         context = example["context"]
#         answers = []

#         # Loop through all features associated with that example
#         for feature_index in example_to_features[example_id]:
#             start_logit = start_logits[feature_index]
#             end_logit = end_logits[feature_index]
#             offsets = features[feature_index]["offset_mapping"]

#             start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
#             end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
#             for start_index in start_indexes:
#                 for end_index in end_indexes:
#                     # Skip answers that are not fully in the context
#                     if offsets[start_index] is None or offsets[end_index] is None:
#                         continue
#                     # Skip answers with a length that is either < 0 or > max_answer_length
#                     if (
#                         end_index < start_index
#                         or end_index - start_index + 1 > max_answer_length
#                     ):
#                         continue

#                     answer = {
#                         "text": context[offsets[start_index][0] : offsets[end_index][1]],
#                         "logit_score": start_logit[start_index] + end_logit[end_index],
#                     }
#                     answers.append(answer)

#         # Select the answer with the best score
#         if len(answers) > 0:
#             best_answer = max(answers, key=lambda x: x["logit_score"])
#             predicted_answers.append(
#                 {"id": example_id, "prediction_text": best_answer["text"]}
#             )
#         else:
#             predicted_answers.append({"id": example_id, "prediction_text": ""})

#     theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
#     return metric.compute(predictions=predicted_answers, references=theoretical_answers)

    

In [24]:
tokenized_dataset['test']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1000
})

In [26]:
import tqdm
model.eval()
eval_loss = 0
eval_steps = 0
start_logits=[]
end_logits=[]
predicted_answers=[]
theoretical_answers=[]

# for step, sample in enumerate(tokenized_dataset['test']):
#tqdm
for step, sample in enumerate(tqdm.tqdm(tokenized_dataset['test'])):
    with torch.no_grad():
        input_ids=torch.tensor([sample['input_ids']]).to('cuda', dtype = torch.long)
        s_embs,e_embs,words_embs = classifier_model(input_ids)
        s_logits,e_logits =get_index(s_embs,e_embs,words_embs,logits_only=True)
        s_logits =s_logits.cpu().numpy()
        e_logits = e_logits.cpu().numpy()
        start_logits.extend(s_logits)
        end_logits.extend(e_logits)

        text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
        start_position = sample['start_positions']
        end_position = sample['end_positions']
        theoretical_answers.append({"id": step, "answers": [{"text": text[start_position:end_position]}]})

        start_position = np.argmax(s_logits)
        end_position = np.argmax(e_logits)
        predicted_answers.append({"id": step, "prediction_text": text[start_position:end_position]})
        


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:43<00:00, 23.09it/s]


In [27]:
import evaluate
# import collections
# import numpy as np

metric = evaluate.load("squad")
metric.compute(predictions=predicted_answers, references=theoretical_answers)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 # import numpy as np                                                                         │
│   4                                                                                              │
│   5 metric = evaluate.load("squad")                                                              │
│ ❱ 6 metric.compute(predictions=predicted_answers, references=theoretical_answers)                │
│   7                                                                                              │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/evaluate/module.py:432 in compute   │
│                                                                                                  │
│   429 │   │   compute_kwargs = {k: kwargs[k] for k in kwargs if k not in self._feature_names()   │
│   430 │   │                                                                                      │
│   431 │   │   if any(v is not None for v in inputs.values()):                                    │
│ ❱ 432 │   │   │   self.add_batch(**inputs)                                                       │
│   433 │   │   self._finalize()                                                                   │
│   434 │   │                                                                                      │
│   435 │   │   self.cache_file_name = None                                                        │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/evaluate/module.py:512 in add_batch │
│                                                                                                  │
│   509 │   │   │   │   │   f"Input predictions: {summarize_if_long_list(predictions)},\n"         │
│   510 │   │   │   │   │   f"Input references: {summarize_if_long_list(references)}"              │
│   511 │   │   │   │   )                                                                          │
│ ❱ 512 │   │   │   raise ValueError(error_msg) from None                                          │
│   513 │                                                                                          │
│   514 │   def add(self, *, prediction=None, reference=None, **kwargs):                           │
│   515 │   │   """Add one prediction and reference for the evaluation module's stack.             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', 
id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': 
Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}},
Input predictions: [{'id': 0, 'prediction_text': ' name? chr'}, {'id': 1, 'prediction_text': ''}, {'id': 2, 
'prediction_text': 'yor'}, ..., {'id': 997, 'prediction_text': 'in'}, {'id': 998, 'prediction_text': 'g '}, {'id': 
999, 'prediction_text': 'nist? '}],
Input references: [{'id': 0, 'answers': [{'text': 'tz was '}]}, {'id': 1, 'answers': [{'text': ''}]}, {'id': 2, 
'answers': [{'text': 'he ne'}]}, ..., {'id': 997, 'answers': [{'text': 'ort'}]}, {'id': 998, 'answers': [{'text': 
'pe of '}]}, {'id': 999, 'answers': [{'text': 'escr'}]}]

### tokenmonster

In [22]:
from datasets import load_dataset

dataset = load_dataset('squad', split='train')
# dataset = dataset.train_test_split(test_size=0.2)
len(dataset)

Found cached dataset squad (/home/kiki/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


87599

In [23]:
dataset[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [24]:
from utils import get_word2subword
def get_answer_span(word2subword_map,answer):
    best_chain=[]
    current_chain=[]
    current_chain_count=0
    for item in word2subword_map:
        word=list(item.keys())[0]
        value=list(item.values())
        if current_chain_count>0:
            if current_chain_count>len(best_chain):
                    best_chain=current_chain
            if word in answer:
                current_chain.append((word,value))
                current_chain_count+=1
            else:
                current_chain=[]
                current_chain_count=0
            
        else:
            if word in answer:
                current_chain.append((word,value))
                current_chain_count+=1
    if len(best_chain)==0:
        return [0,0]
    answer_id_start=best_chain[0][1][0][0]
    answer_id_end=best_chain[-1][1][0][-1]
    if answer_id_start>MODEL_MAX_SEQ_LEN-10:
        answer_id_start=0
        answer_id_end=0
    if answer_id_end>MODEL_MAX_SEQ_LEN -10:
        answer_id_start=0
        answer_id_end=0

    return [answer_id_start,answer_id_end]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 from utils import get_word2subword                                                          │
│    2 def get_answer_span(word2subword_map,answer):                                               │
│    3 │   best_chain=[]                                                                           │
│    4 │   current_chain=[]                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'get_word2subword' from 'utils' 
(/home/kiki/dados_servidor/Servidor/LM_Pretraining/EfficientPretrain/utils/__init__.py)

In [25]:
#operacao dataset["answers"][0] gasta 0.6 segundos. melhor coletar todas as respostas antes
answers=[]
for i in range(len(dataset)):
    answers.append(dataset[i]['answers']['text'][0])
    if i%10000==0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000


In [26]:
# context=dataset["context"][0]
# tokens  = vocab.tokenize(context)
# answer=answers[0]
# t,word2subword_map=get_word2subword(context)
# print(len(tokens),answer)

# best_chain=get_answer_span(word2subword_map,answer)
# best_chain

In [27]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        question=self.dataset["question"][i].lower()+'[SEP]'
        context=self.dataset["context"][i].lower()
        input=norm.normalize_str(question+context)
        answer=norm.normalize_str(answers[i])
        tokens,word2subword_map=get_word2subword(vocab,input)
        label=get_answer_span(word2subword_map,answer)
        label=torch.Tensor(label).long()
                
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()
        
        d={'input_ids':tokens,'label':label}
        return d

from torch.utils.data import DataLoader

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=10
    )

In [28]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [29]:
model_extractor = HiddenLayerExtractor(model.model,-2)

In [30]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.s = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 768)
        self.e = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 768)
        
    
    def forward(self, ids):
        words_embs= self.l1(ids)
        # print('l1:',output.shape)
        output = self.l2(words_embs)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        s_output = self.s(output)
        e_output = self.e(output)
        return s_output,e_output,words_embs

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=5e-5,weight_decay=0.01)

In [31]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [32]:
#FINETUNING

#dot product s_embs(b,768) and e_embs(b,768) with words_embs(b,seq,768) to get index of start and end. Use softmax
def get_index(s_embs,e_embs,words_embs):
    s_index=torch.bmm(words_embs,s_embs.unsqueeze(2)).squeeze(2)
    e_index=torch.bmm(words_embs,e_embs.unsqueeze(2)).squeeze(2)

    s_index=torch.softmax(s_index,dim=1)
    e_index=torch.softmax(e_index,dim=1)

    # s_index=torch.argmax(s_index,dim=1)
    # e_index=torch.argmax(e_index,dim=1)

    return s_index,e_index
    
epochs=5
running_loss=0
for i in range(epochs):
    for j,data in enumerate(train_dataloader):
        # print(j)
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)
        # print('targets: ',targets)

        s_embs,e_embs,words_embs = classifier_model(ids)

        s_index,e_index=get_index(s_embs,e_embs,words_embs)
        # print('s_index: ',s_index)
        # print('e_index: ',e_index)
        start_loss = loss_fn(s_index,targets[:,0])
        end_loss = loss_fn(e_index,targets[:,1])        
        
        loss=(start_loss+end_loss)/2
        running_loss+=loss.item()
        if j%50==0:
            # print(s_index.argmax(dim=1))
            # print(e_index.argmax(dim=1))
            # print(targets[:,0])
            # print(targets[:,1])
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 epochs=5                                                                                    │
│   17 running_loss=0                                                                              │
│   18 for i in range(epochs):                                                                     │
│ ❱ 19 │   for j,data in enumerate(train_dataloader):                                              │
│   20 │   │   # print(j)                                                                          │
│   21 │   │   ids = data['input_ids'].to('cuda', dtype = torch.long)                              │
│   22 │   │   targets = data['label'].to('cuda', dtype = torch.long)                              │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/utils/data/dataloader.py:633  │
│ in __next__                                                                                      │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/utils/data/dataloader.py:677  │
│ in _next_data                                                                                    │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                                                              │
│    679 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    680 │   │   return data                                                                       │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py:51 │
│ in fetch                                                                                         │
│                                                                                                  │
│   48 │   │   │   if hasattr(self.dataset, "__getitems__") and self.dataset.__getitems__:         │
│   49 │   │   │   │   data = self.dataset.__getitems__(possibly_batched_index)                    │
│   50 │   │   │   else:                                                                           │
│ ❱ 51 │   │   │   │   data = [self.dataset[idx] for idx in p

In [33]:
s_index.argmax(dim=1),e_index.argmax(dim=1),targets[:,0],targets[:,1]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 s_index.argmax(dim=1),e_index.argmax(dim=1),targets[:,0],targets[:,1]                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'targets' is not defined

In [34]:
s_index[0]

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.9533e-37, 1.4153e-43, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.5765e-42, 0.0000e+00,
        0.0000e+00, 1.3214e-42, 0.0000e+00, 4.0468e-23, 0.0000e+00, 1.4044e-23,
        3.1101e-09, 2.5207e-15, 1.4376e-20, 7.3730e-08, 1.2263e-09, 5.9128e-13,
        4.1037e-08, 6.9646e-12, 3.4369e-20, 1.0490e-15, 9.4521e-17, 1.0543e-20,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2082e-12, 2.8668e-23,
        2.9138e-22, 0.0000e+00, 3.6187e-28, 0.0000e+00, 1.7171e-22, 4.2841e-24,
        0.0000e+00, 0.0000e+00, 5.9639e-24, 1.9267e-26, 6.0702e-25, 1.3960e-14,
        8.5253e-28, 4.1053e-30, 0.0000e+00, 9.2813e-28, 1.0258e-21, 3.5931e-33,
        3.2127e-22, 1.3028e-35, 7.4481e-

In [35]:
targets.shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 targets.shape                                                                                │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'targets' is not defined

In [36]:
2/0

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 2/0                                                                                          │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ZeroDivisionError: division by zero

In [37]:
#EVALUATE
dataset = load_dataset('glue', task, split='validation')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 #EVALUATE                                                                                    │
│ ❱ 2 dataset = load_dataset('glue', task, split='validation')                                     │
│   3 tokenized_dataset = TokenizedDataset(dataset)                                                │
│   4                                                                                              │
│   5 eval_dataloader = DataLoader(                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'task' is not defined

In [38]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 c=0                                                                                         │
│    5 for _,data in enumerate(eval_dataloader):                                                   │
│    6 │   ids = data['input_ids'].to('cuda', dtype = torch.long)                                  │
│ ❱  7 │   targets = data['label'].to('cuda', dtype = torch.long)                                  │
│    8 │                                                                                           │
│    9 │   outputs = classifier_model(ids)                                                         │
│   10                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'label'

In [39]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 from sklearn.metrics import classification_report                                            │
│ ❱ 2 print(classification_report(labels, predictions))                                            │
│   3                                                                                              │
│                                                                                                  │
│ /home/kiki/miniconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py: │
│ 2363 in classification_report                                                                    │
│                                                                                                  │
│   2360 │   │   │   report_dict[label] = dict(zip(headers, [i.item() for i in scores]))           │
│   2361 │   else:                                                                                 │
│   2362 │   │   longest_last_line_heading = "weighted avg"                                        │
│ ❱ 2363 │   │   name_width = max(len(cn) for cn in target_names)                                  │
│   2364 │   │   width = max(name_width, len(longest_last_line_heading), digits)                   │
│   2365 │   │   head_fmt = "{:>{width}s} " + " {:>9}" * len(headers)                              │
│   2366 │   │   report = head_fmt.format("", *headers, width=width)                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: max() arg is an empty sequence

### Text classification

In [40]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [41]:
model_extractor = HiddenLayerExtractor(model.model,-2)

In [42]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l3 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 3)
    
    def forward(self, ids):
        output= self.l1(ids)
        # print('l1:',output.shape)
        output = self.l2(output)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        output = self.l3(output)
        # print('l3:',output.shape)
        return output

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=2e-5,weight_decay=0.01)

In [43]:
task='mrpc'
dataset = load_dataset('glue', task, split='train')

Found cached dataset glue (/home/kiki/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [44]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.ls=[]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset["sentence1"][i])+'[SEP]'+norm.normalize_str(self.dataset["sentence2"][i])
        tokens = vocab.tokenize(s).tolist()
        
        self.ls.append(i)
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()

        label=self.dataset["label"][i]
        
        d={'input_ids':tokens,'label':label}
        return d



from torch.utils.data import DataLoader

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [45]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [46]:
#FINETUNING

epochs=5
running_loss=0
for i in range(epochs):
    for _,data in enumerate(train_dataloader):
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = classifier_model(ids)
        
        loss = loss_fn(outputs,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if _%50==0:
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Loss:  0.016936131715774537


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   │   # loss=outputs.loss                                                                 │
│   14 │   │                                                                                       │
│   15 │   │   optimizer.zero_grad()                                                               │
│ ❱ 16 │   │   running_loss+=loss.item()                                                           │
│   17 │   │   if _%50==0:                                                                         │
│   18 │   │   │   print(f'Epoch: {i}, Loss:  {running_loss/50}')                                  │
│   19 │   │   │   running_loss=0                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
#EVALUATE
dataset = load_dataset('glue', task, split='validation')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [ ]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.49      0.14      0.22       129
           1       0.70      0.93      0.80       279

    accuracy                           0.68       408
   macro avg       0.59      0.54      0.51       408
weighted avg       0.63      0.68      0.62       408



In [ ]:
# Spanbert 3 layer 3 heads 10000 dataset
#               precision    recall  f1-score   support

#            0       0.38      0.22      0.28       129
#            1       0.70      0.84      0.76       279

#     accuracy                           0.64       408
#    macro avg       0.54      0.53      0.52       408
# weighted avg       0.60      0.64      0.61       408